In [1]:
import numpy as np
from scipy.integrate import fixed_quad
import plotly.graph_objects as go
import pandas as pd
from iminuit import Minuit
from iminuit.cost import LeastSquares
import os 
import matplotlib.pyplot as plt

In [2]:
# Load experimental data
atlas_data = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)
totem_data = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)

# Function to process data for each experiment
def process_data(data, energy_blocks):
    x_values = []
    y_values = []
    y_errors = []
    
    for start, end in energy_blocks:
        block = data.iloc[start:end] if end is not None else data.iloc[start:]
        x_values.append(block[0].values)
        y_values.append(block[1].values)
        y_errors.append(block[2].values)
    
    return x_values, y_values, y_errors

# Energy ranges for each experiment (7TeV, 8TeV, 13TeV)
atlas_blocks = [(0, 29), (29, 58), (58, None)]
totem_blocks = [(0, 65), (65, 118), (118, None)]

# Process data
x_atlas, y_atlas, yerr_atlas = process_data(atlas_data, atlas_blocks)
x_totem, y_totem, yerr_totem = process_data(totem_data, totem_blocks)


# Extract values by energy (index 0=7TeV, 1=8TeV, 2=13TeV)

# ATLAS results
x_7_atlas, y_7_atlas, yerr_7_atlas = x_atlas[0], y_atlas[0], yerr_atlas[0]
x_8_atlas, y_8_atlas, yerr_8_atlas = x_atlas[1], y_atlas[1], yerr_atlas[1]
x_13_atlas, y_13_atlas, yerr_13_atlas = x_atlas[2], y_atlas[2], yerr_atlas[2]

x_7_totem, y_7_totem, yerr_7_totem = x_totem[0], y_totem[0], yerr_totem[0]
x_8_totem, y_8_totem, yerr_8_totem = x_totem[1], y_totem[1], yerr_totem[1]
x_13_totem, y_13_totem, yerr_13_totem = x_totem[2], y_totem[2], yerr_totem[2]


/tmp/ipykernel_36367/3495736505.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  atlas_data = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)
/tmp/ipykernel_36367/3495736505.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  totem_data = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)


In [3]:
n_points = 10000
b_0 = (33 - 6) / (12 * np.pi)
Lambda = 0.284  # ΛQCD in GeV
gamma_1 = 0.084
gamma_2 = 2.36
rho = 4.0
s0 = 1.0
alpha_prime = 0.25

ensemble_parameters = {
    'atlas': {
        'log': {
            'epsilon': 0.0753,
            'mg': 0.356,
            'a1': 1.373,
            'a2': 2.50
        },
        'pl': {
            'epsilon': 0.0753,
            'mg': 0.421,
            'a1': 1.517,
            'a2': 2.05
        }
    },
    'totem': {
        'log': {
            'epsilon': 0.0892,
            'mg': 0.380,
            'a1': 1.491,
            'a2': 2.77
        },
        'pl':{
            'epsilon': 0.0892,
            'mg': 0.447,
            'a1': 1.689,
            'a2': 1.7
        }
    }
}

ensemble_atlas = 'atlas'  
ensemble_totem = 'totem'

log_model_type = 'log'
pl_model_type = 'pl'   


# Get parameters for selected configuration
initial_params_log_atlas = ensemble_parameters[ensemble_atlas][log_model_type]
initial_params_pl_atlas = ensemble_parameters[ensemble_atlas][pl_model_type]

# Create parameter variation ranges (±10%)
initial_params_low_log_atlas = {k: v * 0.70 for k, v in initial_params_log_atlas.items()}
initial_params_high_log_atlas = {k: v * 1.3 for k, v in initial_params_log_atlas.items()}

initial_params_low_pl_atlas = {k: v * 0.70 for k, v in initial_params_pl_atlas.items()}
initial_params_high_pl_atlas = {k: v * 1.3 for k, v in initial_params_pl_atlas.items()}
#-----------------------------------------------------------------------------------------------------------------------

initial_params_log_totem = ensemble_parameters[ensemble_totem][log_model_type]
initial_params_pl_totem = ensemble_parameters[ensemble_totem][pl_model_type]


initial_params_low_log_totem = {k: v * 0.70 for k, v in initial_params_log_totem.items()}
initial_params_high_log_totem = {k: v * 1.3 for k, v in initial_params_log_totem.items()}

initial_params_low_pl_totem = {k: v * 0.70 for k, v in initial_params_pl_totem.items()}
initial_params_high_pl_totem = {k: v * 1.3 for k, v in initial_params_pl_totem.items()}

print("Initial parameters for ATLAS (log):", initial_params_log_atlas)

Initial parameters for ATLAS (log): {'epsilon': 0.0753, 'mg': 0.356, 'a1': 1.373, 'a2': 2.5}


In [4]:
def m2_log(q2, mg):

    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)

    return mg ** 2 * ratio ** (-1 - gamma_1)

def m2_pl(q2, mg):

    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)

    return (mg ** 4 / (q2 + mg ** 2)) * ratio ** (gamma_2 - 1)

def G_p(q2, a1, a2):
    return np.exp(-(a1 * q2 + a2 * q2 ** 2))

def alpha_D(q2, mg, m2_func):
    m2 = m2_func(q2, mg)
    return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

def T_1(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    G0 = G_p(q2, a1, a2)
    return alpha_D_plus * alpha_D_minus * G0 ** 2

def T_2(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    factor = q2 + 9 * abs(k ** 2 - q2 / 4)
    G0 = G_p(q2, a1, a2)
    G_minus = G_p(factor, a1, a2)
    return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)

def integrand(y, x, mg, a1, a2, m2_func, q_val, sqrt_s):
    k = sqrt_s * x 
    phi = 2 * np.pi * y
    jacobian = 2 * np.pi * sqrt_s 
    return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

def amp_calculation(diff_T, s, epsilon, t):
    alpha_pomeron = 1.0 + epsilon + alpha_prime * t
    regge_factor = (s**alpha_pomeron) * 1/(s0**(alpha_pomeron-1))
    return 1j * 8 * regge_factor * diff_T  

def differential_sigma(amp_value, s):
    amp_squared = amp_value.imag * amp_value.imag
    denominator = (16 * np.pi * s**2)
    return amp_squared / denominator * 0.389379323

In [5]:
def model_function_log(x, eps_log, mg_log, a1_log, a2_log, sqrt_s):

    # taking arguments for functions 
    params_log = {
        'epsilon_log': eps_log,
        'mg_log': mg_log,
        'a1_log': a1_log,
        'a2_log': a2_log
    }
    
    dif_sigma_lst_log = []
    
    for q2 in x:
        t = -q2
        
        def inner_integral(x_inner):
            return fixed_quad(
                lambda y: integrand(y, x_inner, params_log['mg_log'], params_log['a1_log'], 
                                  params_log['a2_log'], m2_log, q2, sqrt_s),
                0, 1,
                n=n_points
            )[0]

        integral_value = fixed_quad(
            inner_integral,
            0, 1,
            n=n_points
        )[0]

        diff_T = integral_value
        s = sqrt_s ** 2
        amp_value = amp_calculation(diff_T, s, params_log['epsilon_log'], t)
        dif_sigma_value = differential_sigma(amp_value, s)
        dif_sigma_lst_log.append(dif_sigma_value)
    
    return np.array(dif_sigma_lst_log)

def model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, sqrt_s):

    # taking arguments for functions 
    params_pl = {
        'epsilon_pl': eps_pl,
        'mg_pl': mg_pl,
        'a1_pl': a1_pl,
        'a2_pl': a2_pl
    }
    
    dif_sigma_lst_pl = []
    
    for q2 in x:
        t = -q2
        
        def inner_integral(x_inner):
            return fixed_quad(
                lambda y: integrand(y, x_inner, params_pl['mg_pl'], params_pl['a1_pl'], 
                                  params_pl['a2_pl'], m2_pl, q2, sqrt_s),
                0, 1,
                n=n_points
            )[0]

        integral_value = fixed_quad(
            inner_integral,
            0, 1,
            n=n_points
        )[0]

        diff_T = integral_value
        s = sqrt_s ** 2
        amp_value = amp_calculation(diff_T, s, params_pl['epsilon_pl'], t)
        dif_sigma_value = differential_sigma(amp_value, s)
        dif_sigma_lst_pl.append(dif_sigma_value)
    
    return np.array(dif_sigma_lst_pl)

In [6]:
lsq_7_log_atlas = LeastSquares(x_7_atlas, y_7_atlas, yerr_7_atlas, 
                       lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 7000))

lsq_8_log_atlas = LeastSquares(x_8_atlas, y_8_atlas, yerr_8_atlas, 
                       lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 8000))

lsq_13_log_atlas = LeastSquares(x_13_atlas, y_13_atlas, yerr_13_atlas, 
                        lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 13000))


total_cost_log_atlas = lsq_7_log_atlas + lsq_8_log_atlas + lsq_13_log_atlas

lsq_7_pl_atlas = LeastSquares(x_7_atlas, y_7_atlas, yerr_7_atlas,
                       lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 7000))

lsq_8_pl_atlas = LeastSquares(x_8_atlas, y_8_atlas, yerr_8_atlas,
                       lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 8000))

lsq_13_pl_atlas = LeastSquares(x_13_atlas, y_13_atlas, yerr_13_atlas,
                        lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 13000))

total_cost_pl_atlas = lsq_7_pl_atlas + lsq_8_pl_atlas + lsq_13_pl_atlas

#----------------------------------------------------------------------------------------------------

lsq_7_log_totem = LeastSquares(x_7_totem, y_7_totem, yerr_7_totem, 
                       lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 7000))

lsq_8_log_totem = LeastSquares(x_8_totem, y_8_totem, yerr_8_totem, 
                       lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 8000))

lsq_13_log_totem = LeastSquares(x_13_totem, y_13_totem, yerr_13_totem, 
                        lambda x, eps_log, mg_log, a1_log, a2_log: model_function_log(x, eps_log, mg_log, a1_log, a2_log, 13000))


total_cost_log_totem = lsq_7_log_totem + lsq_8_log_totem + lsq_13_log_totem

lsq_7_pl_totem = LeastSquares(x_7_totem, y_7_totem, yerr_7_totem,
                       lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 7000))

lsq_8_pl_totem = LeastSquares(x_8_totem, y_8_totem, yerr_8_totem,
                       lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 8000))

lsq_13_pl_totem = LeastSquares(x_13_totem, y_13_totem, yerr_13_totem,
                        lambda x, eps_pl, mg_pl, a1_pl, a2_pl: model_function_pl(x, eps_pl, mg_pl, a1_pl, a2_pl, 13000))

total_cost_pl_totem = lsq_7_pl_totem + lsq_8_pl_totem + lsq_13_pl_totem

In [7]:
print("Iniciando otimização dos parâmetros usando LeastSquares...")

# Cria o objeto Minuit com o custo combinado
m_log_atlas = Minuit(total_cost_log_atlas, 
           eps_log=initial_params_log_atlas['epsilon'],
           mg_log=initial_params_log_atlas['mg'],
           a1_log=initial_params_log_atlas['a1'],
           a2_log=initial_params_log_atlas['a2'])

# Configura os limites (±2% dos valores iniciais)
m_log_atlas.limits['eps_log'] = (initial_params_low_log_atlas['epsilon'], initial_params_high_log_atlas['epsilon'])
m_log_atlas.limits['mg_log'] = (initial_params_low_log_atlas['mg'], initial_params_high_log_atlas['mg'])
m_log_atlas.limits['a1_log'] = (initial_params_low_log_atlas['a1'], initial_params_high_log_atlas['a1'])
m_log_atlas.limits['a2_log'] = (initial_params_low_log_atlas['a2'], initial_params_high_log_atlas['a2'])

# Configurações adicionais
# m.strategy = 2
m_log_atlas.errordef = 1
m_log_atlas.tol = 0.001


m_log_atlas.migrad()
# m.simplex()
m_log_atlas.hesse()
m_log_atlas.minos(cl=0.9)
m_log_atlas.migrad()

/home/victorli/miniconda3/lib/python3.12/site-packages/iminuit/minuit.py:139: ErrordefAlreadySetWarning: cost function has an errordef attribute equal to 1.0, you should not override this with Minuit.errordef
  warnings.warn(msg, ErrordefAlreadySetWarning)


Iniciando otimização dos parâmetros usando LeastSquares...


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 24.04 (χ²/ndof = 0.2)      │             Nfcn = 2452              │
│ EDM = 2.96e-09 (Goal: 2e-06)     │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ eps_log │  0.0613   │  0.0022   │  -0.0037   │   0.0037   │ 0.05271 │ 0.09789 │       │
│ 1 │ mg_log  │   0.334   │   0.004   │   -0.006   │   0.007    │ 0.2492  │ 0.4628  │       │
│ 2 │ a1_log  │   1.60    │   0.05    │   -0.05    │    0.07    │ 0.9611  │ 1.7849  │       │
│ 3 │ a2_log  │   3.04    │   0.30    │   -0.54    │    0.21    │  1.75   │  3.25   │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┐
│          │        eps_log        │        mg_log         │        a1_log         │        a2_log         │
├──────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┤
│  Error   │  -0.004   │   0.004   │  -0.006   │   0.007   │   -0.05   │   0.07    │   -0.54   │   0.21    │
│  Valid   │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │
│ At Limit │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   True    │
│ Max FCN  │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │
│ New Min  │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │
└──────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┘
┌─────────┬─────────────────────────────────────┐
│         │  eps_log   mg_log   a1_log   a2_log │
├─────────┼─────────────────────────────────────┤
│ eps_log │ 4.98e-06     8e-6    38e-6  -148e-6 │
│  mg_log │     8e-6 1.55e-05 0.022e-3 0.070e-3 │
│  a1_log │    38e-6 0.022e-3  0.00207  -0.0138 │
│  a2_log │  -148e-6 0.070e-3  -0.0138    0.103 │
└─────────┴─────────────────────────────────────┘

In [ ]:
print("Iniciando otimização dos parâmetros pl atlas usando LeastSquares...")
m_pl_atlas = Minuit(total_cost_pl_atlas,
           eps_pl=initial_params_pl_atlas['epsilon'],
           mg_pl=initial_params_pl_atlas['mg'],
           a1_pl=initial_params_pl_atlas['a1'],
           a2_pl=initial_params_pl_atlas['a2'])

m_pl_atlas.limits['eps_pl'] = (initial_params_low_pl_atlas['epsilon'], initial_params_high_pl_atlas['epsilon'])
m_pl_atlas.limits['mg_pl'] = (initial_params_low_pl_atlas['mg'], initial_params_high_pl_atlas['mg'])
m_pl_atlas.limits['a1_pl'] = (initial_params_low_pl_atlas['a1'], initial_params_high_pl_atlas['a1'])
m_pl_atlas.limits['a2_pl'] = (initial_params_low_pl_atlas['a2'], initial_params_high_pl_atlas['a2'])

m_pl_atlas.errordef = 1
m_pl_atlas.tol = 0.001

m_pl_atlas.migrad()
# m.simplex()
m_pl_atlas.hesse()
m_pl_atlas.minos(cl=0.9)
m_pl_atlas.migrad()

In [ ]:
print("Iniciando otimização dos parâmetros log totem usando LeastSquares...")

# Cria o objeto Minuit com o custo combinado
m_log_totem = Minuit(total_cost_log_totem, 
           eps_log=initial_params_log_totem['epsilon'],
           mg_log=initial_params_log_totem['mg'],
           a1_log=initial_params_log_totem['a1'],
           a2_log=initial_params_log_totem['a2'])

# Configura os limites (±2% dos valores iniciais)
m_log_totem.limits['eps_log'] = (initial_params_low_log_totem['epsilon'], initial_params_high_log_totem['epsilon'])
m_log_totem.limits['mg_log'] = (initial_params_low_log_totem['mg'], initial_params_high_log_totem['mg'])
m_log_totem.limits['a1_log'] = (initial_params_low_log_totem['a1'], initial_params_high_log_totem['a1'])
m_log_totem.limits['a2_log'] = (initial_params_low_log_totem['a2'], initial_params_high_log_totem['a2'])

# Configurações adicionais
# m.strategy = 2
m_log_totem.errordef = 1
m_log_totem.tol = 0.001

m_log_totem.migrad()
# m.simplex()
m_log_totem.hesse()
m_log_totem.minos(cl=0.9)
m_log_totem.migrad()

In [ ]:
print("Iniciando otimização dos parâmetros pl totem usando LeastSquares...")
m_pl_totem = Minuit(total_cost_pl_totem,
           eps_pl=initial_params_pl_totem['epsilon'],
           mg_pl=initial_params_pl_totem['mg'],
           a1_pl=initial_params_pl_totem['a1'],
           a2_pl=initial_params_pl_totem['a2'])

m_pl_totem.limits['eps_pl'] = (initial_params_low_pl_totem['epsilon'], initial_params_high_pl_totem['epsilon'])
m_pl_totem.limits['mg_pl'] = (initial_params_low_pl_totem['mg'], initial_params_high_pl_totem['mg'])
m_pl_totem.limits['a1_pl'] = (initial_params_low_pl_totem['a1'], initial_params_high_pl_totem['a1'])
m_pl_totem.limits['a2_pl'] = (initial_params_low_pl_totem['a2'], initial_params_high_pl_totem['a2'])

m_pl_totem.errordef = 1
m_pl_totem.tol = 0.001

m_pl_totem.migrad()
# m.simplex()
m_pl_totem.hesse()
m_pl_totem.minos(cl=0.9)
m_pl_totem.migrad()

In [ ]:
# print("Parâmetros otimizados:")
# print(f"eps_log: {m_log_atlas.values['eps_log']:.4f} ± {m_log_atlas.errors['eps_log']:.4f}")
# print(f"mg_log: {m_log_atlas.values['mg_log']:.4f} ± {m_log_atlas.errors['mg_log']:.4f}")
# print(f"a1_log: {m_log_atlas.values['a1_log']:.4f} ± {m_log_atlas.errors['a1_log']:.4f}")
# print(f"a2_log: {m_log_atlas.values['a2_log']:.4f} ± {m_log_atlas.errors['a2_log']:.4f}")

# print(f"eps_pl: {m_pl_atlas.values['eps_pl']:.4f} ± {m_pl_atlas.errors['eps_pl']:.4f}")
# print(f"mg_pl: {m_pl_atlas.values['mg_pl']:.4f} ± {m_pl_atlas.errors['mg_pl']:.4f}")
# print(f"a1_pl: {m_pl_atlas.values['a1_pl']:.4f} ± {m_pl_atlas.errors['a1_pl']:.4f}")
# print(f"a2_pl: {m_pl_atlas.values['a2_pl']:.4f} ± {m_pl_atlas.errors['a2_pl']:.4f}")

In [12]:
# print("Parâmetros otimizados:")
# print(f"eps_log: {m_log_totem.values['eps_log']:.4f} ± {m_log_totem.errors['eps_log']:.4f}")
# print(f"mg_log: {m_log_totem.values['mg_log']:.4f} ± {m_log_totem.errors['mg_log']:.4f}")
# print(f"a1_log: {m_log_totem.values['a1_log']:.4f} ± {m_log_totem.errors['a1_log']:.4f}")
# print(f"a2_log: {m_log_totem.values['a2_log']:.4f} ± {m_log_totem.errors['a2_log']:.4f}")

# print(f"eps_pl: {m_pl_totem.values['eps_pl']:.4f} ± {m_pl_totem.errors['eps_pl']:.4f}")
# print(f"mg_pl: {m_pl_totem.values['mg_pl']:.4f} ± {m_pl_totem.errors['mg_pl']:.4f}")
# print(f"a1_pl: {m_pl_totem.values['a1_pl']:.4f} ± {m_pl_totem.errors['a1_pl']:.4f}")
# print(f"a2_pl: {m_pl_totem.values['a2_pl']:.4f} ± {m_pl_totem.errors['a2_pl']:.4f}")

In [13]:
# dat_atlas = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)

# lst_x_atlas = []
# lst_y_atlas = []
# lst_error_atlas = []

# # Extracting data for ATLAS
# data_blocks_atlas = [
#     (0, 29),   # dsig 7000 (linhas 0-28)
#     (29, 58),  # dsig 8000 (linhas 29-57)
#     (58, None) # dsig 13000 (linhas 58 até final)
# ]

# for start, end in data_blocks_atlas:
#     if end is not None:
#         df = dat_atlas.iloc[start:end]
#     else:
#         df = dat_atlas.iloc[start:]
    
#     lst_x_atlas.append(df[0].to_numpy())
#     lst_y_atlas.append(df[1].to_numpy())
#     lst_error_atlas.append(df[2].to_numpy())

# x_7000_atlas, y_7000_atlas, y_error_7000_atlas = lst_x_atlas[0], lst_y_atlas[0], lst_error_atlas[0]
# x_8000_atlas, y_8000_atlas, y_error_8000_atlas = lst_x_atlas[1], lst_y_atlas[1], lst_error_atlas[1]
# x_13000_atlas, y_13000_atlas, y_error_13000_atlas = lst_x_atlas[2], lst_y_atlas[2], lst_error_atlas[2]


# # === Global Configuration and Constants ===
# start_q2 = 0.006  # Start of q2 range
# max_q2 = 0.204   # End of q2 range
# q2_step = 0.001

# b_0 = (33 - 6) / (12 * np.pi)  # β0 for nf=3
# Lambda = 0.284  # ΛQCD in GeV
# gamma_1 = 0.084
# gamma_2 = 2.36
# rho = 4.0

# epsilon_log_atlas = m_log_atlas.values['eps_log']
# epsilon_pl_atlas = m_pl_atlas.values['eps_pl']

# dif_sigma_lst_log_atlas = []
# dif_sigma_lst_pl_atlas = []  


# q2_lst_log_atlas = []
# q2_lst_pl_atlas = []

# error_lst_log_atlas = []
# error_lst_pl_atlas = []

# s0 = 1.0 
# alpha_prime = 0.25
# sqrt_s = 7000



# model_params = {
#     'atlas': {
#         'log': {'mg': m_log_atlas.values['mg_log'], 
#                 'a1': m_log_atlas.values['a1_log'], 
#                 'a2': m_log_atlas.values['a2_log']
#                 },
#         'pl': {'mg': m_pl_atlas.values['mg_pl'],
#                'a1': m_pl_atlas.values['a1_pl'],
#                'a2': m_pl_atlas.values['a2_pl']},
#     }
# }


# epsilon_values_atlas = {
#     'log_atlas': epsilon_log_atlas,
#     'pl_atlas': epsilon_pl_atlas
# }

# # === Auxiliary Functions for Physical Model ===
# def save_results(filename, q2_val, diff_T, real_part, im_part, sigma_tot):
#     header = not os.path.exists(filename) or os.path.getsize(filename) == 0
#     with open(filename, 'a') as file:
#         if header:
#             file.write(f"{'q2':<15} {'Diff_T':<15} {'Real part':<15} {'Imag part':<15} {'Sigma_Tot':<15}\n")
#         file.write(f"{q2_val:<15.6f} {diff_T:<15.8f} {real_part:<15.8f} {im_part:<15.8f} {sigma_tot:<15.8f}\n")

# def m2_log(q2, mg):
#     lambda_squared = Lambda ** 2
#     rho_mg_squared = rho * mg ** 2
#     ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
#     return mg ** 2 * ratio ** (-1 - gamma_1)

# def get_m2_function(mass_model):
#     return m2_log if mass_model == 'log' else m2_pl

# def G_p(q2, a1, a2):
#     return np.exp(-(a1 * q2 + a2 * q2 ** 2))

# def alpha_D(q2, mg, m2_func):
#     m2 = m2_func(q2, mg)
#     return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

# def T_1(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q 
#     qk_cos = np.sqrt(q) * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
#     G0 = G_p(q2, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G0 ** 2

# def T_2(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q 
#     qk_cos = np.sqrt(q) * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)

#     factor = q2 + 9 * abs(k ** 2 - q2 / 4)
    
#     G0 = G_p(q2, a1, a2)
#     G_minus = G_p(factor, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)

# def integrand(y, x, mg, a1, a2, m2_func, q_val):
#     k = sqrt_s * x 
#     phi = 2 * np.pi * y
#     jacobian = 2 * np.pi * sqrt_s 
#     return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

# def amp_calculation(diff_T, s, epsilon, t):
#     alpha_pomeron = 1.0 + epsilon + alpha_prime * t
#     regge_factor = (s**alpha_pomeron) * 1/(s0**(alpha_pomeron-1))
#     return 1j * 8 * regge_factor * diff_T  

# def sigma_tot(amp_value, s):
#     return amp_value.imag / s * 0.389379323

# def differential_sigma(amp_value, s):
#     amp_squared = amp_value.imag * amp_value.imag
#     denominator  =  (16 * np.pi * s**2)
#     return  amp_squared / denominator * 0.389379323

# # === Main Function ===
# def main():
#     mass_model_log = 'log'
#     mass_model_pl = 'pl'

#     ensemble = 'atlas'  # Change to 'atlas' or 'totem' as needed
#     n_points = 10000

#     m2_func_log = get_m2_function(mass_model_log)
#     m2_func_pl = get_m2_function(mass_model_pl)

#     epsilon_log_atlas = epsilon_values_atlas['log_atlas']
#     epsilon_pl_atlas = epsilon_values_atlas['pl_atlas']

#     mg_log_atlas = model_params[ensemble]['log']['mg']
#     a1_log_atlas = model_params[ensemble]['log']['a1']
#     a2_log_atlas = model_params[ensemble]['log']['a2']

#     mg_pl_atlas = model_params[ensemble]['pl']['mg']
#     a1_pl_atlas = model_params[ensemble]['pl']['a1']
#     a2_pl_atlas = model_params[ensemble]['pl']['a2']



#     sqrt_s_values = [7000, 8000, 13000]
#     scale_factors = {7000: 1, 8000: 10, 13000: 100}

#     fig = go.Figure()  # Criar figura do Plotly

#     for sqrt_s in sqrt_s_values:
#         scale = scale_factors[sqrt_s]

#         dif_sigma_lst_log_atlas = []
#         dif_sigma_lst_pl_atlas = []

#         q2_lst_log_atlas = []
#         q2_lst_pl_atlas = []

#         print(f"=== Starting calculation for sqrt(s) = {sqrt_s/1000} TeV ===")

#         q2 = start_q2 
#         while q2 <= max_q2: 
#             t = -q2

#             def inner_integral_log_atlas(x):
#                 return fixed_quad(
#                     lambda y: integrand(y, x, mg_log_atlas, a1_log_atlas, a2_log_atlas, m2_func_log, q2),
#                     0, 1,
#                     n=n_points
#                 )[0]

#             integral_value_log_atlas = fixed_quad(
#                 inner_integral_log_atlas,
#                 0, 1,
#                 n=n_points
#             )[0]

#             diff_T_log_atlas = integral_value_log_atlas
#             s = sqrt_s ** 2
#             amp_value_log_atlas = amp_calculation(diff_T_log_atlas, s, epsilon_log_atlas, t)
#             dif_sigma_value_log_atlas = differential_sigma(amp_value_log_atlas, s) * scale
#             dif_sigma_lst_log_atlas.append(dif_sigma_value_log_atlas)
#             q2_lst_log_atlas.append(q2)

#             def inner_integral_pl_atlas(x):
#                 return fixed_quad(
#                     lambda y: integrand(y, x, mg_pl_atlas, a1_pl_atlas, a2_pl_atlas, m2_func_pl, q2),
#                     0, 1,
#                     n=n_points
#                 )[0]

#             integral_value_pl_atlas = fixed_quad(
#                 inner_integral_pl_atlas,
#                 0, 1,
#                 n=n_points
#             )[0]

#             diff_T_pl_atlas = integral_value_pl_atlas
#             s = sqrt_s ** 2
#             amp_value_pl_atlas = amp_calculation(diff_T_pl_atlas, s, epsilon_pl_atlas, t)
#             dif_sigma_value_pl_atlas = differential_sigma(amp_value_pl_atlas, s) * scale
#             dif_sigma_lst_pl_atlas.append(dif_sigma_value_pl_atlas)
#             q2_lst_pl_atlas.append(q2)
            
#             q2 += q2_step

#         print(f"=== Completed for sqrt(s) = {sqrt_s/1000} TeV ===")

#         # Adicionar curvas para o modelo log
#         fig.add_trace(go.Scatter(
#             x=q2_lst_log_atlas,
#             y=dif_sigma_lst_log_atlas,
#             mode='lines+markers',
#             line=dict(color='red'),
#             name=f'log √s={sqrt_s//1000} TeV ×{scale}'
#         ))

#         fig.add_trace(go.Scatter(
#             x=q2_lst_pl_atlas,
#             y=dif_sigma_lst_pl_atlas,
#             mode='lines+markers',
#             line=dict(color='blue'),
#             name=f'pl √s={sqrt_s//1000} TeV ×{scale}'
#         ))

    

#         # Adicionar dados experimentais
#         if ensemble == 'atlas':
#             show_label = True
#             if sqrt_s == 7000:
#                 fig.add_trace(go.Scatter(
#                     x=x_7000_atlas,
#                     y=y_7000_atlas * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_7000_atlas * scale,
#                         visible=True
#                     ),
#                     name='ATLAS (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False
#             elif sqrt_s == 8000:
#                 fig.add_trace(go.Scatter(
#                     x=x_8000_atlas,
#                     y=y_8000_atlas * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_8000_atlas * scale,
#                         visible=True
#                     ),
#                     name='ATLAS (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False
#             elif sqrt_s == 13000:
#                 fig.add_trace(go.Scatter(
#                     x=x_13000_atlas,
#                     y=y_13000_atlas * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_13000_atlas * scale,
#                         visible=True
#                     ),
#                     name='ATLAS (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False

#     # Configurar layout do gráfico
#     fig.update_layout(
#         title=f'Differential cross section vs. |t| for log and pl models ({ensemble.upper()})',
#         xaxis_title='|t| (GeV²)',
#         yaxis_title='dσ/dt (mb/GeV²)',
#         yaxis_type='log',
#         legend_title='Model',
#         plot_bgcolor='white',
#         hovermode='x unified'
#     )
    
#     # Adicionar anotações para os fatores de escala
#     fig.add_annotation(
#         x=0.01, y=10**3+1500,
#         text="(10x)",
#         showarrow=False,
#         font=dict(size=12)
#     )
#     fig.add_annotation(
#         x=0.01, y=10**4+17500,
#         text="(100x)",
#         showarrow=False,
#         font=dict(size=12)
#     )
    
#     fig.update_xaxes(gridcolor='lightgray')
#     fig.update_yaxes(gridcolor='lightgray')

#     # Mostrar o gráfico
#     fig.show(renderer='browser')
#     # fig.write_html(f"results/dif_sigma/dif_sigma_{ensemble}_with_data.html")
#     # fig.write_image(f"results/dif_sigma/dif_sigma_{ensemble}_with_data.pdf", width=1200, height=600)

# if __name__ == "__main__":
#     main()

In [14]:
# dat_totem = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)

# lst_x_totem = []
# lst_y_totem = []
# lst_error_totem = []

# # Extracting data for TOTEM
# data_blocks_totem = [
#     (0, 65),   # dsig 7000 (linhas 0-28)
#     (65, 118),  # dsig 8000 (linhas 29-57)
#     (118, None) # dsig 13000 (linhas 58 até final)
# ]

# for start, end in data_blocks_totem:
#     if end is not None:
#         df = dat_totem.iloc[start:end]
#     else:
#         df = dat_totem.iloc[start:]
    
#     lst_x_totem.append(df[0].to_numpy())
#     lst_y_totem.append(df[1].to_numpy())
#     lst_error_totem.append(df[2].to_numpy())

# x_7000_totem, y_7000_totem, y_error_7000_totem = lst_x_totem[0], lst_y_totem[0], lst_error_totem[0]
# x_8000_totem, y_8000_totem, y_error_8000_totem = lst_x_totem[1], lst_y_totem[1], lst_error_totem[1]
# x_13000_totem, y_13000_totem, y_error_13000_totem = lst_x_totem[2], lst_y_totem[2], lst_error_totem[2]


# # === Global Configuration and Constants ===
# start_q2 = 0.006  # Start of q2 range
# max_q2 = 0.204   # End of q2 range
# q2_step = 0.001

# b_0 = (33 - 6) / (12 * np.pi)  # β0 for nf=3
# Lambda = 0.284  # ΛQCD in GeV
# gamma_1 = 0.084
# gamma_2 = 2.36
# rho = 4.0

# epsilon_log_totem = m_log_totem.values['eps_log']
# epsilon_pl_totem = m_pl_totem.values['eps_pl']

# dif_sigma_lst_log_totem = []
# dif_sigma_lst_pl_totem = []  

# q2_lst_log_totem = []
# q2_lst_pl_totem = []

# error_lst_log_totem = []
# error_lst_pl_totem = []

# s0 = 1.0 
# alpha_prime = 0.25
# sqrt_s = 7000

# model_params = {
#     'totem': {
#         'log': {'mg': m_log_totem.values['mg_log'], 
#                 'a1': m_log_totem.values['a1_log'], 
#                 'a2': m_log_totem.values['a2_log']
#                 },
#         'pl': {'mg': m_pl_totem.values['mg_pl'],
#                'a1': m_pl_totem.values['a1_pl'],
#                'a2': m_pl_totem.values['a2_pl']},
#     }
# }

# epsilon_values_totem = {
#     'log_totem': epsilon_log_totem,
#     'pl_totem': epsilon_pl_totem
# }

# # === Auxiliary Functions for Physical Model ===
# def save_results(filename, q2_val, diff_T, real_part, im_part, sigma_tot):
#     header = not os.path.exists(filename) or os.path.getsize(filename) == 0
#     with open(filename, 'a') as file:
#         if header:
#             file.write(f"{'q2':<15} {'Diff_T':<15} {'Real part':<15} {'Imag part':<15} {'Sigma_Tot':<15}\n")
#         file.write(f"{q2_val:<15.6f} {diff_T:<15.8f} {real_part:<15.8f} {im_part:<15.8f} {sigma_tot:<15.8f}\n")

# def m2_log(q2, mg):
#     lambda_squared = Lambda ** 2
#     rho_mg_squared = rho * mg ** 2
#     ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
#     return mg ** 2 * ratio ** (-1 - gamma_1)

# def get_m2_function(mass_model):
#     return m2_log if mass_model == 'log' else m2_pl

# def G_p(q2, a1, a2):
#     return np.exp(-(a1 * q2 + a2 * q2 ** 2))

# def alpha_D(q2, mg, m2_func):
#     m2 = m2_func(q2, mg)
#     return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

# def T_1(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q 
#     qk_cos = np.sqrt(q) * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
#     G0 = G_p(q2, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G0 ** 2

# def T_2(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q 
#     qk_cos = np.sqrt(q) * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)

#     factor = q2 + 9 * abs(k ** 2 - q2 / 4)
    
#     G0 = G_p(q2, a1, a2)
#     G_minus = G_p(factor, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)

# def integrand(y, x, mg, a1, a2, m2_func, q_val):
#     k = sqrt_s * x 
#     phi = 2 * np.pi * y
#     jacobian = 2 * np.pi * sqrt_s 
#     return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

# def amp_calculation(diff_T, s, epsilon, t):
#     alpha_pomeron = 1.0 + epsilon + alpha_prime * t
#     regge_factor = (s**alpha_pomeron) * 1/(s0**(alpha_pomeron-1))
#     return 1j * 8 * regge_factor * diff_T  

# def sigma_tot(amp_value, s):
#     return amp_value.imag / s * 0.389379323

# def differential_sigma(amp_value, s):
#     amp_squared = amp_value.imag * amp_value.imag
#     denominator  =  (16 * np.pi * s**2)
#     return  amp_squared / denominator * 0.389379323

# # === Main Function ===
# def main():
#     mass_model_log = 'log'
#     mass_model_pl = 'pl'

#     ensemble = 'totem'  # Changed to 'totem'
#     n_points = 10000

#     m2_func_log = get_m2_function(mass_model_log)
#     m2_func_pl = get_m2_function(mass_model_pl)

#     epsilon_log_totem = epsilon_values_totem['log_totem']
#     epsilon_pl_totem = epsilon_values_totem['pl_totem']

#     mg_log_totem = model_params[ensemble]['log']['mg']
#     a1_log_totem = model_params[ensemble]['log']['a1']
#     a2_log_totem = model_params[ensemble]['log']['a2']

#     mg_pl_totem = model_params[ensemble]['pl']['mg']
#     a1_pl_totem = model_params[ensemble]['pl']['a1']
#     a2_pl_totem = model_params[ensemble]['pl']['a2']

#     sqrt_s_values = [7000, 8000, 13000]
#     scale_factors = {7000: 1, 8000: 10, 13000: 100}

#     fig = go.Figure()  # Criar figura do Plotly

#     for sqrt_s in sqrt_s_values:
#         scale = scale_factors[sqrt_s]

#         dif_sigma_lst_log_totem = []
#         dif_sigma_lst_pl_totem = []

#         q2_lst_log_totem = []
#         q2_lst_pl_totem = []

#         print(f"=== Starting calculation for sqrt(s) = {sqrt_s/1000} TeV ===")

#         q2 = start_q2 
#         while q2 <= max_q2: 
#             t = -q2

#             def inner_integral_log_totem(x):
#                 return fixed_quad(
#                     lambda y: integrand(y, x, mg_log_totem, a1_log_totem, a2_log_totem, m2_func_log, q2),
#                     0, 1,
#                     n=n_points
#                 )[0]

#             integral_value_log_totem = fixed_quad(
#                 inner_integral_log_totem,
#                 0, 1,
#                 n=n_points
#             )[0]

#             diff_T_log_totem = integral_value_log_totem
#             s = sqrt_s ** 2
#             amp_value_log_totem = amp_calculation(diff_T_log_totem, s, epsilon_log_totem, t)
#             dif_sigma_value_log_totem = differential_sigma(amp_value_log_totem, s) * scale
#             dif_sigma_lst_log_totem.append(dif_sigma_value_log_totem)
#             q2_lst_log_totem.append(q2)

#             def inner_integral_pl_totem(x):
#                 return fixed_quad(
#                     lambda y: integrand(y, x, mg_pl_totem, a1_pl_totem, a2_pl_totem, m2_func_pl, q2),
#                     0, 1,
#                     n=n_points
#                 )[0]

#             integral_value_pl_totem = fixed_quad(
#                 inner_integral_pl_totem,
#                 0, 1,
#                 n=n_points
#             )[0]

#             diff_T_pl_totem = integral_value_pl_totem
#             s = sqrt_s ** 2
#             amp_value_pl_totem = amp_calculation(diff_T_pl_totem, s, epsilon_pl_totem, t)
#             dif_sigma_value_pl_totem = differential_sigma(amp_value_pl_totem, s) * scale
#             dif_sigma_lst_pl_totem.append(dif_sigma_value_pl_totem)
#             q2_lst_pl_totem.append(q2)
            
#             q2 += q2_step

#         print(f"=== Completed for sqrt(s) = {sqrt_s/1000} TeV ===")

#         # Adicionar curvas para o modelo log
#         fig.add_trace(go.Scatter(
#             x=q2_lst_log_totem,
#             y=dif_sigma_lst_log_totem,
#             mode='lines+markers',
#             line=dict(color='red'),
#             name=f'log √s={sqrt_s//1000} TeV ×{scale}'
#         ))

#         fig.add_trace(go.Scatter(
#             x=q2_lst_pl_totem,
#             y=dif_sigma_lst_pl_totem,
#             mode='lines+markers',
#             line=dict(color='blue'),
#             name=f'pl √s={sqrt_s//1000} TeV ×{scale}'
#         ))

#         # Adicionar dados experimentais
#         if ensemble == 'totem':
#             show_label = True
#             if sqrt_s == 7000:
#                 fig.add_trace(go.Scatter(
#                     x=x_7000_totem,
#                     y=y_7000_totem * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_7000_totem * scale,
#                         visible=True
#                     ),
#                     name='TOTEM (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False
#             elif sqrt_s == 8000:
#                 fig.add_trace(go.Scatter(
#                     x=x_8000_totem,
#                     y=y_8000_totem * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_8000_totem * scale,
#                         visible=True
#                     ),
#                     name='TOTEM (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False
#             elif sqrt_s == 13000:
#                 fig.add_trace(go.Scatter(
#                     x=x_13000_totem,
#                     y=y_13000_totem * scale,
#                     mode='markers',
#                     marker=dict(color='black', size=6),
#                     error_y=dict(
#                         type='data',
#                         array=y_error_13000_totem * scale,
#                         visible=True
#                     ),
#                     name='TOTEM (exp)' if show_label else None,
#                     showlegend=show_label
#                 ))
#                 show_label = False

#     # Configurar layout do gráfico
#     fig.update_layout(
#         title=f'Differential cross section vs. |t| for log and pl models (TOTEM)',
#         xaxis_title='|t| (GeV²)',
#         yaxis_title='dσ/dt (mb/GeV²)',
#         yaxis_type='log',
#         legend_title='Model',
#         plot_bgcolor='white',
#         hovermode='x unified'
#     )
    
#     # Adicionar anotações para os fatores de escala
#     fig.add_annotation(
#         x=0.01, y=10**3+1500,
#         text="(10x)",
#         showarrow=False,
#         font=dict(size=12)
#     )
#     fig.add_annotation(
#         x=0.01, y=10**4+17500,
#         text="(100x)",
#         showarrow=False,
#         font=dict(size=12)
#     )
    
#     fig.update_xaxes(gridcolor='lightgray')
#     fig.update_yaxes(gridcolor='lightgray')

#     # Mostrar o gráfico
#     fig.show(renderer='browser')
#     # fig.write_html(f"results/dif_sigma/dif_sigma_totem_with_data.html")
#     # fig.write_image(f"results/dif_sigma/dif_sigma_totem_with_data.pdf", width=1200, height=600)

# if __name__ == "__main__":
#     main()

In [15]:
# data_atlas = pd.read_csv(
#     "../../data/sigma_tot/ensemble_atlas.dat",
#     delim_whitespace=True,
#     header=None,
#     nrows=69  # lê apenas as 70 primeiras linhas
# )

# x_atlas = data_atlas[0].to_numpy()
# y_atlas = data_atlas[1].to_numpy()
# y_error_atlas = data_atlas[2].to_numpy()

# start_sqrt_s = 1

# fig = go.Figure() 

# def m2_log(q2, mg):
#     lambda_squared = Lambda ** 2
#     rho_mg_squared = rho * mg ** 2
#     ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
#     return mg ** 2 * ratio ** (-1 - gamma_1)

# def m2_pl(q2, mg):
#     lambda_squared = Lambda ** 2
#     rho_mg_squared = rho * mg ** 2
#     ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
#     return (mg ** 4 / (q2 + mg ** 2)) * ratio ** (gamma_2 - 1)

# def get_m2_function(mass_model):
#     return m2_log if mass_model == 'log' else m2_pl

# def G_p(q2, a1, a2):
#     return np.exp(-(a1 * q2 + a2 * q2 ** 2))

# def alpha_D(q2, mg, m2_func):
#     m2 = m2_func(q2, mg)
#     return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

# def T_1(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q ** 2
#     qk_cos = q * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
#     G0 = G_p(q2, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G0 ** 2

# def T_2(k, q, phi, mg, a1, a2, m2_func):
#     q2 = q ** 2
#     qk_cos = q * k * np.cos(phi)
#     qk_plus_squared = q2 / 4 + qk_cos + k ** 2
#     qk_minus_squared = q2 / 4 - qk_cos + k ** 2

#     alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
#     alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)

#     factor = q2 + 9 * abs(k ** 2 - q2 / 4)

#     G0 = G_p(q2, a1, a2)
#     G_minus = G_p(factor, a1, a2)

#     return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)


# def integrand(y, x, mg, a1, a2, m2_func):
#     k = sqrt_s * x
#     phi = 2 * np.pi * y
#     jacobian = 2 * np.pi * sqrt_s

#     return k * (T_1(k, 0.0, phi, mg, a1, a2, m2_func) - T_2(k, 0.0, phi, mg, a1, a2, m2_func)) * jacobian

# def amp_calculation(diff_T, s, epsilon):
#     alpha_pomeron = 1.0 + epsilon
#     regge_factor = (s / s0) ** alpha_pomeron
    
#     return 1j * 8.0 * regge_factor * diff_T

# def sigma_tot(amp_value, s):
#     return amp_value.imag / s * 0.389379323



# sigma_tot_lst_log_atlas = []
# sigma_tot_lst_pl_atlas = []

# sqrt_s_lst = []

# def main():
#     global start_sqrt_s
#     global sqrt_s
#     global sigma_tot_lst_log_atlas
#     global sigma_tot_lst_pl_atlas
#     global sqrt_s_lst

#     max_sqrt_s = 13000
#     step = 100
#     n_points = 10000

#     # Limpe as listas antes de começar
#     sigma_tot_lst_log_atlas.clear()
#     sigma_tot_lst_pl_atlas.clear()

#     sqrt_s_lst.clear()

#     mass_model_log_atlas = 'log'  
#     mass_model_pl_atlas = 'pl'

#     m2_func_log_atlas = get_m2_function(mass_model_log_atlas)
#     m2_func_pl_atlas = get_m2_function(mass_model_pl_atlas)

#     mg_log_atlas, a1_log_atlas, a2_log_atlas = m_log_atlas.values['mg_log'], m_log_atlas.values['a1_log'], m_log_atlas.values['a2_log']
#     mg_pl_atlas, a1_pl_atlas, a2_pl_atlas = m_pl_atlas.values['mg_pl'], m_pl_atlas.values['a1_pl'], m_pl_atlas.values['a2_pl']

#     epsilon_log_atlas = epsilon_values_atlas['log_atlas']
#     epsilon_pl_atlas = epsilon_values_atlas['pl_atlas']

#     sqrt_s = start_sqrt_s
#     while sqrt_s <= max_sqrt_s:
#         def inner_integral_log_atlas(x):
#             return fixed_quad(
#                 lambda y: integrand(y, x, mg_log_atlas, a1_log_atlas, a2_log_atlas, m2_func_log_atlas),
#                 0, 1,
#                 n=n_points
#             )[0]

#         integral_value_log_atlas = fixed_quad(
#             inner_integral_log_atlas,
#             0, 1,
#             n=n_points
#         )[0]

#         diff_T_log_atlas = integral_value_log_atlas
#         s = sqrt_s * sqrt_s

#         amp_value_log_atlas = amp_calculation(diff_T_log_atlas, s, epsilon_log_atlas)
#         sigma_tot_value_log_atlas = sigma_tot(amp_value_log_atlas, s)
#         sigma_tot_lst_log_atlas.append(sigma_tot_value_log_atlas)
#         sqrt_s_lst.append(sqrt_s)
        
#         def inner_integral_pl_atlas(x):
#             return fixed_quad(
#                 lambda y: integrand(y, x, mg_pl_atlas, a1_pl_atlas, a2_pl_atlas, m2_func_pl_atlas),
#                 0, 1,
#                 n=n_points
#             )[0]

#         integral_value_pl_atlas = fixed_quad(
#             inner_integral_pl_atlas,
#             0, 1,
#             n=n_points
#         )[0]

#         diff_T_pl_atlas = integral_value_pl_atlas
#         s = sqrt_s * sqrt_s

#         amp_value_pl_atlas = amp_calculation(diff_T_pl_atlas, s, epsilon_pl_atlas)
#         sigma_tot_value_pl_atlas = sigma_tot(amp_value_pl_atlas, s)
#         sigma_tot_lst_pl_atlas.append(sigma_tot_value_pl_atlas)

#         sqrt_s += step
    
#     # Adicionar traço ao gráfico do Plotly
#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst,
#         y=sigma_tot_lst_log_atlas,
#         mode='lines+markers',
#         line=dict(
#             color='red',
#             dash='dash',
#             width=1
#         ),
#         marker=dict(
#             size=3
#         ),
#         name=f'{mass_model_log_atlas} {ensemble_atlas}'
#     ))

#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst,
#         y=sigma_tot_lst_pl_atlas,
#         mode='lines+markers',
#         line=dict(
#             color='blue',
#             dash='dash',
#             width=1
#         ),
#         marker=dict(
#             size=3
#         ),
#         name=f'{mass_model_pl_atlas} {ensemble_atlas}'
#     ))

# # Primeiro chame a função main para calcular os dados
# if __name__ == "__main__":
#     main()

# # Depois adicione os dados do ATLAS
# fig.add_trace(go.Scatter(
#     x=x_atlas,
#     y=y_atlas,
#     mode='markers',
#     marker=dict(
#         color='black',
#         size=6,
#         symbol='square'
#     ),
#     error_y=dict(
#         type='data',
#         array=y_error_atlas,
#         visible=True
#     ),
#     name='ATLAS Data'
# ))

# # Configurar layout do gráfico
# fig.update_layout(
#     title='Sigma Tot vs. sqrt(s) log e pl atlas',
#     xaxis=dict(
#         title='sqrt(s) [GeV]',
#         type='log',
#     ),
#     yaxis=dict(
#         title='Sigma Tot [mb]',
#     ),
#     showlegend=True,
#     legend=dict(
#         title='Modelo'
#     ),
#     plot_bgcolor='white',
#     hovermode='x unified'
# )

# fig.update_xaxes(gridcolor='lightgray')
# fig.update_yaxes(gridcolor='lightgray')

# fig.show(renderer="browser")

In [16]:
# # Leitura dos dados para ATLAS e TOTEM
# data_atlas = pd.read_csv(
#     "../../data/sigma_tot/ensemble_atlas.dat",
#     delim_whitespace=True,
#     header=None,
#     nrows=69
# )

# data_totem = pd.read_csv(
#     "../../data/sigma_tot/ensemble_totem.dat",  # Supondo que existe um arquivo similar para TOTEM
#     delim_whitespace=True,
#     header=None,
#     nrows=84
# )

# x_atlas = data_atlas[0].to_numpy()
# y_atlas = data_atlas[1].to_numpy()
# y_error_atlas = data_atlas[2].to_numpy()

# x_totem = data_totem[0].to_numpy()
# y_totem = data_totem[1].to_numpy()
# y_error_totem = data_totem[2].to_numpy()

# start_sqrt_s = 1
# fig = go.Figure() 

# # [As funções de cálculo permanecem as mesmas...]

# def main():
#     global start_sqrt_s, sqrt_s, sqrt_s_lst
    
#     # Listas para armazenar resultados
#     sigma_tot_lst_log_atlas = []
#     sigma_tot_lst_pl_atlas = []
#     sigma_tot_lst_log_totem = []
#     sigma_tot_lst_pl_totem = []
#     sqrt_s_lst = []

#     max_sqrt_s = 13000
#     step = 100
#     n_points = 10000

#     # Cálculos para ATLAS
#     mass_model_log_atlas = 'log'  
#     mass_model_pl_atlas = 'pl'
#     m2_func_log_atlas = get_m2_function(mass_model_log_atlas)
#     m2_func_pl_atlas = get_m2_function(mass_model_pl_atlas)

#     mg_log_atlas, a1_log_atlas, a2_log_atlas = m_log_atlas.values['mg_log'], m_log_atlas.values['a1_log'], m_log_atlas.values['a2_log']
#     mg_pl_atlas, a1_pl_atlas, a2_pl_atlas = m_pl_atlas.values['mg_pl'], m_pl_atlas.values['a1_pl'], m_pl_atlas.values['a2_pl']
#     epsilon_log_atlas = epsilon_values_atlas['log_atlas']
#     epsilon_pl_atlas = epsilon_values_atlas['pl_atlas']

#     # Cálculos para TOTEM
#     mass_model_log_totem = 'log'
#     mass_model_pl_totem = 'pl'
#     m2_func_log_totem = get_m2_function(mass_model_log_totem)
#     m2_func_pl_totem = get_m2_function(mass_model_pl_totem)

#     mg_log_totem, a1_log_totem, a2_log_totem = m_log_totem.values['mg_log'], m_log_totem.values['a1_log'], m_log_totem.values['a2_log']
#     mg_pl_totem, a1_pl_totem, a2_pl_totem = m_pl_totem.values['mg_pl'], m_pl_totem.values['a1_pl'], m_pl_totem.values['a2_pl']
#     epsilon_log_totem = epsilon_values_totem['log_totem']
#     epsilon_pl_totem = epsilon_values_totem['pl_totem']

#     sqrt_s = start_sqrt_s
#     while sqrt_s <= max_sqrt_s:
#         s = sqrt_s * sqrt_s
        
#         # Cálculos para ATLAS (log)
#         integral_value_log_atlas = fixed_quad(
#             lambda x: fixed_quad(
#                 lambda y: integrand(y, x, mg_log_atlas, a1_log_atlas, a2_log_atlas, m2_func_log_atlas),
#                 0, 1, n=n_points)[0],
#             0, 1, n=n_points)[0]
#         sigma_tot_lst_log_atlas.append(sigma_tot(
#             amp_calculation(integral_value_log_atlas, s, epsilon_log_atlas), s))
        
#         # Cálculos para ATLAS (pl)
#         integral_value_pl_atlas = fixed_quad(
#             lambda x: fixed_quad(
#                 lambda y: integrand(y, x, mg_pl_atlas, a1_pl_atlas, a2_pl_atlas, m2_func_pl_atlas),
#                 0, 1, n=n_points)[0],
#             0, 1, n=n_points)[0]
#         sigma_tot_lst_pl_atlas.append(sigma_tot(
#             amp_calculation(integral_value_pl_atlas, s, epsilon_pl_atlas), s))
        
#         # Cálculos para TOTEM (log)
#         integral_value_log_totem = fixed_quad(
#             lambda x: fixed_quad(
#                 lambda y: integrand(y, x, mg_log_totem, a1_log_totem, a2_log_totem, m2_func_log_totem),
#                 0, 1, n=n_points)[0],
#             0, 1, n=n_points)[0]
#         sigma_tot_lst_log_totem.append(sigma_tot(
#             amp_calculation(integral_value_log_totem, s, epsilon_log_totem), s))
        
#         # Cálculos para TOTEM (pl)
#         integral_value_pl_totem = fixed_quad(
#             lambda x: fixed_quad(
#                 lambda y: integrand(y, x, mg_pl_totem, a1_pl_totem, a2_pl_totem, m2_func_pl_totem),
#                 0, 1, n=n_points)[0],
#             0, 1, n=n_points)[0]
#         sigma_tot_lst_pl_totem.append(sigma_tot(
#             amp_calculation(integral_value_pl_totem, s, epsilon_pl_totem), s))
        
#         sqrt_s_lst.append(sqrt_s)
#         sqrt_s += step

#     # Adicionar todos os traços ao gráfico
#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst, y=sigma_tot_lst_log_atlas,
#         mode='lines+markers',
#         line=dict(color='red', dash='dash', width=1),
#         marker=dict(size=3),
#         name='Log ATLAS'
#     ))

#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst, y=sigma_tot_lst_pl_atlas,
#         mode='lines+markers',
#         line=dict(color='blue', dash='dash', width=1),
#         marker=dict(size=3),
#         name='PL ATLAS'
#     ))

#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst, y=sigma_tot_lst_log_totem,
#         mode='lines+markers',
#         line=dict(color='red', dash='dot', width=1),
#         marker=dict(size=3),
#         name='Log TOTEM'
#     ))

#     fig.add_trace(go.Scatter(
#         x=sqrt_s_lst, y=sigma_tot_lst_pl_totem,
#         mode='lines+markers',
#         line=dict(color='blue', dash='dot', width=1),
#         marker=dict(size=3),
#         name='PL TOTEM'
#     ))

#     # Adicionar dados experimentais
#     fig.add_trace(go.Scatter(
#         x=x_atlas, y=y_atlas,
#         mode='markers',
#         marker=dict(color='black', size=6, symbol='square'),
#         error_y=dict(type='data', array=y_error_atlas, visible=True),
#         name='ATLAS Data'
#     ))

#     fig.add_trace(go.Scatter(
#         x=x_totem, y=y_totem,
#         mode='markers',
#         marker=dict(color='black', size=6, symbol='circle'),
#         error_y=dict(type='data', array=y_error_totem, visible=True),
#         name='TOTEM Data'
#     ))

#     # Configurar layout
#     fig.update_layout(
#         title='Comparação de Sigma Tot vs. sqrt(s) para ATLAS e TOTEM',
#         xaxis=dict(title='sqrt(s) [GeV]', type='log'),
#         yaxis=dict(title='Sigma Tot [mb]'),
#         showlegend=True,
#         legend=dict(title='Modelo'),
#         plot_bgcolor='white',
#         hovermode='x unified'
#     )

#     fig.update_xaxes(gridcolor='lightgray')
#     fig.update_yaxes(gridcolor='lightgray')

# if __name__ == "__main__":
#     main()
#     fig.show(renderer="browser")